In [1]:
import os

import numpy as np
from statistics import mean
from decimal import Decimal
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import matplotlib.patches as patches
import bamt.networks as Nets
import bamt.preprocessors as pp
import bamt
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from pgmpy.estimators import K2Score
from bamt.builders import StructureBuilder
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import r2_score
import math
import random
from scipy.spatial.distance import cdist
from numpy import array
from pgmpy.utils import get_example_model
from pgmpy.estimators import PC
from pgmpy.estimators import K2Score
from sklearn import preprocessing
from pgmpy.estimators import K2Score


from sklearn.model_selection import train_test_split
from tqdm import tqdm
from joblib import Parallel, delayed
from pyvis.network import Network
from pyitlib import discrete_random_variable as drv
from pgmpy.estimators import K2Score
import bamt.builders as builders
import itertools 
from itertools import islice

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
import numpy as np
from bamt.networks.continuous_bn import ContinuousBN
from bamt.networks.hybrid_bn import HybridBN

In [3]:
def covert_to_boolean(a):
    return [bool(x) for x in a]

In [21]:
ends=['_Laptevyh_k001.csv','_Laptevyh_k003.csv','_Laptevyh_k005.csv','_Laptevyh_k01.csv']

In [4]:
ends=['_Laptevyh_const1030.csv','_Laptevyh_const50100.csv','_Laptevyh_const100200.csv']

In [4]:
ends=['_Laptevyh_sigma2.csv','_Laptevyh_sigma10.csv','_Laptevyh_sigma50.csv']

In [5]:
f1=[]
for end in ends:
    ZIMA = pd.read_csv(f'Зима2019.npy{end}')
    VESNA = pd.read_csv(f'Весна2019.npy{end}')
    LETO = pd.read_csv(f'Лето2019.npy{end}')
    OSEN = pd.read_csv(f'Осень2019.npy{end}')
    
    ZIMA =ZIMA.drop('Unnamed: 0', axis=1)
    VESNA =VESNA.drop('Unnamed: 0', axis=1)
    LETO =LETO.drop('Unnamed: 0', axis=1)
    OSEN =OSEN.drop('Unnamed: 0', axis=1)
    
    df = pd.concat([ZIMA, VESNA, LETO, OSEN], ignore_index= True)
    
    df=df.astype({'NORM': 'float64'})
    
    bn = HybridBN(use_mixture=True, has_logit=True)
    
    encoder = preprocessing.LabelEncoder()
    discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

    p = pp.Preprocessor([('encoder', encoder), ('discretizer', discretizer)])
    discretized_data, est = p.apply(df)

    info = p.info

    bn.add_nodes(info)

    structure = [("VALUE", "NORM"),
            ("SEASON", "NORM")]

    bn.set_structure(edges=structure)
    
    data_train, data_test = train_test_split(df, test_size=0.3, random_state=42)

    target = data_test['NORM'].values

    data_test = data_test.drop(columns=['NORM'])
    
    bn.fit_parameters(data_train)
    
    predictions = bn.predict(data_test, 4)
    
    pred= covert_to_boolean(predictions['NORM'])
    
    f1.append(f1_score(target,pred, average='macro'))
    


100%|█████████████████████████████████████████████████████████████████████████████| 5460/5460 [00:06<00:00, 841.24it/s]


In [6]:
f1

[0.6031590694480213, 0.635832505225645, 0.48938557935097726]

In [23]:
f1

[0.49811563562827466, 0.5807756499108936, 0.620076454394823, 0.473430417590896]

In [19]:
f1

[0.5035269926271284, 0.49250677080814437, 0.6948999671808509]

In [13]:
df=df.astype({'NORM': 'object'})

In [10]:
df[df['NORM']==True]

,VALUE,NORM,SEASON
0,98.890000,True,winter
1,97.600000,True,winter
34,100.000000,True,winter
35,100.000000,True,winter
36,98.479996,True,winter
...,...,...,...
18195,99.290000,True,autumn
18196,99.670000,True,autumn
18197,99.930000,True,autumn
18198,99.320000,True,autumn


In [15]:
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [24]:
df

,VALUE,NORM,SEASON
0,64.500000,True,winter
1,75.340000,True,winter
2,64.210000,True,winter
3,91.360000,True,winter
4,93.060000,True,winter
...,...,...,...
69995,73.549995,True,autumn
69996,0.000000,True,autumn
69997,0.000000,True,autumn
69998,0.000000,True,autumn


In [11]:
df.dtypes

VALUE     float64
NORM         bool
SEASON     object
dtype: object